$$\textrm{Joaquin Peñuela Parraz}$$
$$\textrm{University of Los Andes}$$
$$\textrm{High Energy Physics Group: Phenomenology of Particles}$$

This code was written to be running in Docker. If you do not have a Docker inside hep-server2 please refer to: https://github.com/Phenomenology-group-uniandes/Tutoriales_Generales

In [1]:
import os, sys

def add_parent_lib_path(name="Semanas_07_08_09"):
    sys.path.append(os.path.join(sys.path[0].split(name)[0],name))
    
add_parent_lib_path()

In [2]:
Path_Leptoquarks_searches = os.path.dirname(os.path.dirname(os.path.realpath('XBG_Discriminator_Histograms.ipynb')))

In [3]:
from Uniandes_Framework.ml_tools import tools
from Uniandes_Framework.delphes_reader import root_analysis

import pandas as pd

from ROOT import TCanvas #Permite poner los histogramas
from ROOT import THStack #Permite graficar varios histogramas al mismo tiempo
from ROOT import TLegend #Permite poner legends cuando se sobrelapan histogramas
from ROOT import TLatex #Permite poner avisos en Latex en las graficas

Welcome to JupyROOT 6.22/06


In [4]:
channels = ["hadronic_dLQ", "hadronic_sLQ", "hadronic_non-resonant", "semileptonic_dLQ", "semileptonic_sLQ", "semileptonic_non-resonant"]
signals = ['LQ_LQ', 'Tau_LQ', 'Tau_Tau']
Masses = ['1250', '1500', '1750', '2000', '2250', '2500']
bkgs = ['ttbar', 'z_jets', 'w_jets', 'stop','ww', 'wz', 'zz']

In [5]:
try: os.mkdir('Histograms')
except: pass

for channel in channels:
    try: os.mkdir(f'Histograms/{channel}')
    except: pass    

    bkg_dict = {}
    for bkg in bkgs: bkg_dict[bkg] = os.path.join(os.sep,"disco4","pheno_csv_files","Leptoquarks_Searches",bkg,f"{bkg}_{channel}.csv")
    
    most_important_features = []
    with open(os.path.join(os.getcwd(), 'XGB_models', channel, f'Most_Important_Features.txt'), "r") as f:
        for line in f: most_important_features.append(line.strip())
    f.close()
    
    for Mass in Masses:
        try: os.mkdir(f'Histograms/{channel}/M{Mass}')
        except: pass

        signal_dict = {}
        for signal in signals: signal_dict[f"{signal}_{Mass}"] = os.path.join(os.sep,"disco4","pheno_csv_files","Leptoquarks_Searches",f"{signal}_{Mass}",f"{signal}_{Mass}_{channel}.csv")

        path_model_mass =  os.path.join(os.getcwd(), 'XGB_models', channel, f'M{Mass}_XGB.joblib')
        bkg_and_signal_dict = bkg_dict | signal_dict
        path_folder_mass = os.path.join(os.getcwd(), 'Histograms', channel,f'M{Mass}')
        
        with root_analysis.Quiet(): tools.hist_discriminator(path_model= path_model_mass, csv_dict = bkg_and_signal_dict, path_to_save= path_folder_mass, best_features= most_important_features)